# Generating Word Embedding Models using Gensim, Exploratory Data Analysis Notebook

In [1]:
sys.path.append(r"C:\Users\robal\Dropbox\UChicago\Year II\Quarter III\Advanced Machine Learning\Project\GenderProject")

In [45]:
import pandas as pd 
import numpy as np 
import gensim 
import spacy 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
## NN based spanish sentiment scorer built on Tensorflow + Keras
from sentiment_analysis_spanish import sentiment_analysis
from pre_process.text_utils import *

nlp_spanish = spacy.load("es_core_news_sm")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Use La Jornada web scraped articles on gender violence which contains around ~2000 articles

In [4]:
jornada_df = pd.read_excel("../data/Mexico/la_jornada_scraped.xlsx")

In [5]:
jornada_df.head()

,Unnamed: 0,link,datetime,title,text,authors,summary,keywords,images,has_top_image,top_image,successfully_scraped,is_valid_url,meta_lang,meta_data,media_links
0,0,https://www.jornada.com.mx/ultimas/2020/03/04/...,2020-03-04,"Implementar política de género, labor de la Co...",A la nueva Coordinación de Igualdad de Género ...,['Arturo Sánchez Jiménez'],"La Coordinación de Igualdad de Género, que dep...","['género', 'y', 'una', 'la', 'los', 'unam', 'l...",{'https://media.jornada.com.mx/ultimas/2021/04...,1.0,https://www.jornada.com.mx/ultimas/2020/03/04/...,1,NaN,NaN,NaN,NaN
1,1,https://www.jornada.com.mx/ultimas/2020/08/06/...,2020-08-06,Arrestan a productor ganador del Grammy por ca...,El productor musical ganador del Grammy Noel F...,[],El productor musical ganador del Grammy Noel F...,"['cargos', 'y', 'la', 'arrestan', 'los', 'víct...",{'https://media.jornada.com.mx/ultimas/2021/04...,1.0,https://www.jornada.com.mx/ultimas/2020/08/06/...,1,NaN,NaN,NaN,NaN
2,2,https://www.jornada.com.mx/ultimas/2020/04/14/...,2020-04-14,El cine realizado con 'smartphone' se abre cam...,"En la primavera de 2019, el escritor y product...",[],"En la primavera de 2019, el escritor y product...","['se', 'y', 'una', 'la', 'abre', 'es', 'smartp...",{'https://media.jornada.com.mx/ultimas/2021/04...,1.0,https://www.jornada.com.mx/ultimas/2020/04/14/...,1,NaN,NaN,NaN,NaN
3,3,https://www.jornada.com.mx/ultimas/2021/03/19/...,2021-03-19,Otorgan dos suspensiones definitivas contra Le...,Ciudad de México. El Juzgado Segundo de Distri...,"['Editorial Demos', 'Gustavo Castillo García']",Con esta resolución y las impugnaciones realiz...,"['contra', 'el', 'se', 'la', 'y', 'los', 'defi...",{'https://www.jornada.com.mx/ultimas/2021/03/2...,1.0,https://www.jornada.com.mx/ultimas/2021/03/19/...,1,NaN,NaN,NaN,NaN
4,4,https://www.jornada.com.mx/ultimas/2021/02/17/...,2021-02-17,Conceden tres nuevas suspensiones contra Tren ...,El Juzgado Cuarto de Distrito con sede en Yuca...,"['Editorial Demos', 'Angélica Enciso Y Jared L...",El Juzgado Cuarto de Distrito con sede en Yuca...,"['contra', 'conceden', 'maya', 'la', 'se', 'y'...",{'https://www.jornada.com.mx/theme/img/logo_ne...,1.0,https://www.jornada.com.mx/ultimas/2021/02/17/...,1,NaN,NaN,NaN,NaN


In [15]:
jornada_df["title"]

0       Implementar política de género, labor de la Co...
1       Arrestan a productor ganador del Grammy por ca...
2       El cine realizado con 'smartphone' se abre cam...
3       Otorgan dos suspensiones definitivas contra Le...
4       Conceden tres nuevas suspensiones contra Tren ...
                              ...                        
2094    Secuestrador pretendía abusar de Sofía Abigail...
2095    "Desde nosotras", la mirada fotográfica de las...
2096    Con Motín, presos de Bolivia exigen “indulto t...
2097    Claman justicia en Michoacán por el asesinato ...
2098    Fernández acusa a empresas de causar desabasto...
Name: title, Length: 2099, dtype: object

In [27]:
# Join both text columns
jornada_df["title"] = jornada_df["title"].str.pad(1, side ='right', fillchar =' ')
jornada_df["complete_text"]  = jornada_df.title.str.cat(jornada_df.text).astype(str)

In [30]:
# Normalize and tokenize all documents
jornada_df['tokenized_sentences'] = jornada_df['complete_text'].apply(lambda x: 
                                                                      [word_tokenize(s, model=nlp_spanish)
                                                                      for s in sent_tokenize(x)])
jornada_df['normalized_sentences'] = jornada_df['tokenized_sentences'].apply(lambda x:
                                                                            [normalize_tokens(s, model=nlp_spanish)
                                                                            for s in x])

In [31]:
laJornada_w2v  = gensim.models.word2vec.Word2Vec(jornada_df['normalized_sentences'].sum(),window=10)

In [46]:
# Initialize sentiment scorer object
SentimentScorer = sentiment_analysis.SentimentAnalysisSpanish()

C:\Users\robal\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 0.24.0. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\robal\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.23.2 when using version 0.24.0. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [38]:
laJornada_w2v.wv.most_similar("hombre")

[('expareja', 0.9249137043952942),
 ('parejo', 0.9185882806777954),
 ('joven', 0.914275050163269),
 ('ileso', 0.9124696850776672),
 ('huella', 0.9100757837295532),
 ('hallar', 0.9099434018135071),
 ('edad', 0.9064306616783142),
 ('morir', 0.9054591655731201),
 ('agredir', 0.8991708755493164),
 ('vehículo', 0.8988486528396606)]

In [64]:
words_of_interest = ["hombre", "mujer", "feminicidio", "Morena"]

for word in words_of_interest:
    print()
    print(f"Finding words most similar to '{word}' according to its embedding")
    try:
        print("................................................................")
        words_tupple = laJornada_w2v.wv.most_similar(word)
        rep_words = [word for word, score in words_tupple]
        print(words_tupple)
        sentiment = SentimentScorer.sentiment(" ".join(rep_words))
        print(f"Sentiment {sentiment}")
    except:
        continue


Finding words most similar to 'hombre' according to its embedding
................................................................
[('expareja', 0.9249137043952942), ('parejo', 0.9185882806777954), ('joven', 0.914275050163269), ('ileso', 0.9124696850776672), ('huella', 0.9100757837295532), ('hallar', 0.9099434018135071), ('edad', 0.9064306616783142), ('morir', 0.9054591655731201), ('agredir', 0.8991708755493164), ('vehículo', 0.8988486528396606)]
Sentiment 0.5502848660732251

Finding words most similar to 'mujer' according to its embedding
................................................................
[('machista', 0.8625805974006653), ('doméstico', 0.8457156419754028), ('vivir', 0.8359159231185913), ('feminicida', 0.834751307964325), ('erradicar', 0.830609917640686), ('intrafamiliar', 0.8279011249542236), ('familiar', 0.8210567235946655), ('estudiante', 0.8158153891563416), ('aumentar', 0.8135995864868164), ('contexto', 0.8109303712844849)]
Sentiment 0.544059203489677

Finding word

## Use El Universal web scraped articles on gender violence which contains around ~3000 articles

In [67]:
# Missing